In [24]:
#import all needed libs and set base paths for the data
import pandas as pd, numpy as np, tempfile
from pathlib import Path
from tqdm.notebook import tqdm

base_path = Path()
raw_data_path = base_path.joinpath('raw-data')
processed_data_path = base_path.joinpath('processed-data')
split_data_path = base_path.joinpath('split-data')

In [34]:
#All functions for handling the data are defined here

def load_data(type:str):
    '''type: raw, processed, or split \n
    returns a dataframe of all the data from the specified type'''
    type = type.lower()
    try:
        if type == 'raw':
            all_files = raw_data_path.glob('*.csv')
        elif type == 'processed':
            all_files = processed_data_path.glob('*.csv')
        elif type == 'split':
            all_files = split_data_path.glob('*.csv')
        all_files = [filename.as_posix() for filename in all_files]

        if len(list(all_files)) == 0:
            print('No files found in raw data folder')
            return None
        elif len(list(all_files)) == 1:
            data = pd.read_csv(all_files[0], encoding_errors='ignore', header=0, low_memory=False)
        elif len(list(all_files)) > 1:
            data = pd.concat([pd.read_csv(f, encoding_errors='ignore', header=0, low_memory=False) for f in all_files])
        return data
    except Exception as e:
        print(e)
        return None

def save_data(data: pd.DataFrame, max_file_size: int, processed: bool = True):
    '''Takes in a dataframe and splits it into smaller dataframes of the specified size in MB if needed.\n
    If save is set to True, the dataframes will be saved. \n
    If processed is set to True, the dataframes will be saved to the processed-data folder. Otherwise they will be saved to the split-data folder.'''
    
    def estimate_csv_size(df):
        # Serialize the DataFrame to a temporary file and check its size
        with tempfile.NamedTemporaryFile() as temp_csv:
            df.to_csv(temp_csv, index=False)
            return temp_csv.tell() / 1024 ** 2

    data_size = estimate_csv_size(data)
    if processed:
        save_path = processed_data_path
    else:
        save_path = split_data_path

    for data_file in tqdm(save_path.glob('*.csv'), f'Deleting old files from {"Processed" if processed else "Split"} Data folder', total=len(list(save_path.glob('*.csv')))):
        data_file.unlink()

    if data_size < max_file_size:
        print(f'Data has a size of {data_size:.2f}MB which is already smaller than {max_file_size}MB')
        data.to_csv(save_path.joinpath('0.csv'), index=False)
    else:
        num_files = np.ceil(data_size / max_file_size)
        print(f'Data is {data_size:.2f}MB, splitting into {num_files} files')

        # Split the data into multiple DataFrames
        split_data = np.array_split(data, num_files)

        for i, df in tqdm(enumerate(split_data), f'Saving data to {"Processed" if processed else "Split"} Data folder', total=len(split_data)):
            df.to_csv(save_path.joinpath(f'{i}.csv'), index=False)

    return True

def prune_col(data):
    '''Takes in a dataframe and removes all unnecessary columns.\n
    Returns the dataframe with the columns removed.'''
    keep_columns = [
        "BankName", "GrossApproval", "ApprovalDate", "ApprovalFiscalYear", "InitialInterestRate", "TermInMonths", "BusinessType", "BusinessAge", "LoanStatus",
        "PaidInFullDate", "GrossChargeOffAmount", "RevolverStatus", "JobsSupported"
    ]
    data = data[keep_columns]
    return data

def fill_missing(data:pd.DataFrame):
    '''Takes in a pruned dataframe and fills some of the columns with estimated values. \n
    Returns the dataframe with the columns filled. '''

    data['BusinessType'].fillna('Individual', inplace=True)
    data['GrossApproval'].fillna(0, inplace=True)
    

    min_intrest_map = data.loc[:,['ApprovalFiscalYear', 'InitialInterestRate']].dropna().groupby('ApprovalFiscalYear').mean().to_dict()['InitialInterestRate']
    min_year = data['ApprovalFiscalYear'].min()
    max_year = data['ApprovalFiscalYear'].max()
    for year in range(min_year, max_year + 1):
        if year not in min_intrest_map:
            min_intrest_map[year] = min_intrest_map[min(min_intrest_map.keys(), key=lambda k: abs(k-year))]
    min_intrest_map = dict(sorted(min_intrest_map.items()))
    data['InitialInterestRate'].fillna(data['ApprovalFiscalYear'].map(min_intrest_map), inplace=True)
    data['InitialInterestRate'] = data['InitialInterestRate'].round(2)
    
    buisness_age_matrix = {
        'Existing or more than 2 years old': 2,
        'Startup, Loan Funds will Open Business': 0,
        'New Business or 2 years or less': 1,
        'Change of Ownership': 2,
        'Unanswered': 1,
        'Existing, 5 or more years': 5,
        'New, Less than 1 Year old': 1,
        'Less than 3 years old but at least 2': 2,
        'Less than 4 years old but at least 3': 3,
        'Less than 5 years old but at least 4': 4
    }
    data['BusinessAge'].fillna('Unanswered', inplace=True)
    data['BusinessAge'] = data['BusinessAge'].map(buisness_age_matrix)

    return data


In [37]:
raw_data = load_data('raw')
save_data(raw_data, 85, processed=False)

Deleting old files from Split Data folder:   0%|          | 0/6 [00:00<?, ?it/s]

Data is 584.30MB, splitting into 7.0 files


Saving data to Split Data folder:   0%|          | 0/7 [00:00<?, ?it/s]

True

In [6]:
pruned_data = prune_col(raw_data)
pruned_data = fill_missing(pruned_data)
print(pruned_data.shape)
display(pruned_data.head())

C:\Users\sebas\AppData\Local\Temp\ipykernel_19208\1102838769.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BusinessType'].fillna('Individual', inplace=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_19208\1102838769.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['GrossApproval'].fillna(0, inplace=True)
C:\Users\sebas\AppData\Local\Temp\ipykernel_19208\1102838769.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

(1755610, 13)


C:\Users\sebas\AppData\Local\Temp\ipykernel_19208\1102838769.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BusinessAge'] = data['BusinessAge'].map(buisness_age_matrix)


,BankName,GrossApproval,ApprovalDate,ApprovalFiscalYear,InitialInterestRate,TermInMonths,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,GrossChargeOffAmount,RevolverStatus,JobsSupported
0,CIBM Bank,330000.0,10/01/1990,1991,6.54,84,CORPORATION,1,PIF,07/31/2005,0,0,0
1,BMO Harris Bank National Association,98900.0,10/01/1990,1991,6.54,11,CORPORATION,1,PIF,07/31/2005,0,0,0
2,MISSINGMAINBANKID,120000.0,10/01/1990,1991,6.54,240,CORPORATION,1,CANCLD,NaN,0,0,0
3,"U.S. Bank, National Association",250000.0,10/01/1990,1991,6.54,48,CORPORATION,1,PIF,07/31/2005,0,0,0
4,Manufacturers and Traders Trust Company,33500.0,10/01/1990,1991,6.54,84,CORPORATION,1,PIF,07/31/2005,0,0,0


In [7]:
#get percentage of rows since 2018
print(pruned_data[pruned_data['ApprovalFiscalYear'] >= 2018].shape[0]/pruned_data.shape[0])

0.1678789708420435


In [8]:
#get percentage of revolving credit lines
print(pruned_data[pruned_data['RevolverStatus'] == 1].shape[0]/pruned_data.shape[0])

0.2525327379087611


In [9]:
#get each columns percentage of null values
null_percent = pruned_data.isnull().sum() / len(pruned_data)
null_percent = null_percent[null_percent > 0]
null_percent.sort_values(inplace=True)
null_percent

PaidInFullDate    0.395203
dtype: float64

In [20]:
recast_dict = {
    'BankName': 'category',
    'GrossApproval': 'int32',
    'ApprovalDate': 'datetime64[ns]',
    'ApprovalFiscalYear': 'int16',
    'InitialInterestRate': 'float32',
    'TermInMonths': 'int16',
    'BusinessType': 'category',
    'BusinessAge': 'int8',
    'LoanStatus': 'category',
    'PaidInFullDate': 'datetime64[ns]',
    'GrossChargeOffAmount': 'int32',
    'RevolverStatus': 'int8',
    'JobsSupported': 'int32'
}
pruned_data = pruned_data.astype(recast_dict)
display(pruned_data.dtypes)
display(pruned_data.memory_usage(deep=True))

BankName                      category
GrossApproval                    int32
ApprovalDate            datetime64[ns]
ApprovalFiscalYear               int16
InitialInterestRate            float32
TermInMonths                     int16
BusinessType                  category
BusinessAge                       int8
LoanStatus                    category
PaidInFullDate          datetime64[ns]
GrossChargeOffAmount             int32
RevolverStatus                    int8
JobsSupported                    int32
dtype: object

Index                   14044880
BankName                 4071719
GrossApproval            7022440
ApprovalDate            14044880
ApprovalFiscalYear       3511220
InitialInterestRate      7022440
TermInMonths             3511220
BusinessType             1756052
BusinessAge              1755610
LoanStatus               1756094
PaidInFullDate          14044880
GrossChargeOffAmount     7022440
RevolverStatus           1755610
JobsSupported            7022440
dtype: int64

In [39]:
save_data(pruned_data, 85, processed=True)

Deleting old files from Processed Data folder:   0%|          | 0/1 [00:00<?, ?it/s]

Data is 155.66MB, splitting into 2.0 files


Saving data to Processed Data folder:   0%|          | 0/2 [00:00<?, ?it/s]

True

In [ ]:
#print summary statistics for some numeric columns
pruned_data.describe().apply(lambda s: s.apply(lambda x: format(x, 'f'))).drop(['ApprovalFiscalYear', 'NaicsCode', 'BusinessAge', 'RevolverStatus'], axis=1)

,GrossApproval,InitialInterestRate,TermInMonths,GrossChargeOffAmount,JobsSupported
count,1755610.000000,1755610.000000,1755610.000000,1755610.000000,1755610.000000
mean,286517.849603,6.577298,114.743535,12360.987425,7.788571
std,536508.776978,1.155749,77.998548,72501.106151,27.514746
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35000.000000,6.350000,63.000000,0.000000,0.000000
50%,100000.000000,6.540000,84.000000,0.000000,3.000000
75%,299500.000000,6.540000,120.000000,0.000000,8.000000
max,5000000.000000,56.000000,847.000000,4706180.000000,9500.000000
